In [1]:
import dateparser
import numpy as np
import pandas as pd

tables = pd.read_html("https://en.wikipedia.org/wiki/Northern_Ireland_Executive")
tables[4].columns = range(len(tables[4].columns))

# Get rid of the nasty multi index
executive_events = tables[4][[0, 1, 2, 3, 4, 5, 6]]
executive_events.columns = [
    "Executive",
    "Date",
    "Event",
    "vFM",
    "FM",
    "vDFM",
    "DFM",
]

# Get rid of the comments row at the bottom
executive_events = executive_events[:-1]

# Clean up the 'Executive' as for some reason wikipedians count the caretakers differently.
executive_events["Executive"] = executive_events["Executive"].map(
    lambda x: x.split("(")[0].split(' ')[0] # Such that 5th E.C. (6th A.) -> 5th
)

# Use the OFMDFM posts as a proxy for 'active' to flatten out the range of reasons for failure.
executive_events["Active"] = (
    executive_events[["vFM", "FM", "vDFM", "DFM"]]
    .replace("Vacant", None)
    .replace(np.nan, None)
    .any(axis=1)
)

executive_durations = (
    executive_events.groupby(["Executive", "Active"])["Date"].first().unstack()
)

executive_durations.columns = ["Dissolved", "Established"]
executive_durations = executive_durations[reversed(executive_durations.columns)]

# map in pandas 2.10; applymap in other versions
executive_durations.fillna('').applymap(
    lambda x: dateparser.parse(x) if x else None
).diff(axis=1).iloc[:, -1:]

/var/folders/9p/y8jwvy1n7v10llkj6r9qtdrc0000gn/T/ipykernel_9419/3148849025.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  executive_durations.fillna('').applymap(


,Dissolved
Executive,
1st,1566 days
2nd,1416 days
3rd,1827 days
4th,235 days
5th,754 days
6th,NaT


In [2]:

executive_durations = executive_durations.applymap(
    lambda s: dateparser.parse(s) if isinstance(s, str) else s
)
executive_durations["Duration"] = executive_durations.diff(axis=1).iloc[:, -1:]

executive_dissolutions = pd.concat(
    [
        executive_durations["Dissolved"],
        executive_durations["Established"].shift(-1),
    ],
    axis=1,
)


/var/folders/9p/y8jwvy1n7v10llkj6r9qtdrc0000gn/T/ipykernel_9419/1783321512.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  executive_durations = executive_durations.applymap(


In [3]:

import datetime
executive_dissolutions = executive_dissolutions.apply(
    lambda r: r.Established - r.Dissolved
    if not pd.isnull(r.Established)
    else datetime.datetime.today().replace(
        hour=0, minute=0, second=0, microsecond=0
    )
    - r.Dissolved,
    axis=1,
)

executive_durations["Interregnum"] = executive_dissolutions

In [4]:
# Fix last / most recent

executive_durations['Duration'].iloc[-1] = datetime.datetime.today().replace(
    hour=0, minute=0, second=0, microsecond=0
) - executive_durations['Established'].iloc[-1]
executive_durations

/var/folders/9p/y8jwvy1n7v10llkj6r9qtdrc0000gn/T/ipykernel_9419/3501325718.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  executive_durations['Duration'].iloc[-1] = datetime.datetime.today().replace(
/var/folders/9p/y8jwvy1n7v10llkj6r9q

,Established,Dissolved,Duration,Interregnum
Executive,,,,
1st,1998-07-01,2002-10-14,1566 days,1667 days
2nd,2007-05-08,2011-03-24,1416 days,53 days
3rd,2011-05-16,2016-05-16,1827 days,10 days
4th,2016-05-26,2017-01-16,235 days,1090 days
5th,2020-01-11,2022-02-03,754 days,730 days
6th,2024-02-03,NaT,219 days,NaT


In [5]:
executive_durations.dtypes

Established     datetime64[ns]
Dissolved       datetime64[ns]
Duration       timedelta64[ns]
Interregnum    timedelta64[ns]
dtype: object

In [11]:
executive_durations['Interregnum'].sum()

Timedelta('3550 days 00:00:00')

In [ ]:
executive_durations